In [1]:
# Metric Copy from implement by 
# https://www.kaggle.com/c/tensorflow-great-barrier-reef/discussion/290083
# https://www.kaggle.com/bamps53/competition-metric-implementation?scriptVersionId=81087805

# Yolo infer modified from 
# https://www.kaggle.com/awsaf49/great-barrier-reef-yolov5-infer

In [13]:
import os

from pathlib import Path
import glob
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 30, 30
np.set_printoptions(precision=3, suppress=True)
from ast import literal_eval
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
from norfair import Detection, Tracker
import os
import cv2
import matplotlib.pyplot as plt
import glob
import shutil
import sys
sys.path.append('../input/tensorflow-great-barrier-reef')
import torch
from PIL import Image
import ast
import timm
import torch
from torch import nn
import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations import *


%matplotlib inline

# Utils func

In [3]:
def get_path(row):
    row['image_path'] = f'{ROOT_DIR}/train_images/video_{row.video_id}/{row.video_frame}.jpg'
    return row

def voc2yolo(bboxes, image_height=720, image_width=1280):
    """
    voc  => [x1, y1, x2, y2]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]/ image_height
    
    w = bboxes[..., 2] - bboxes[..., 0]
    h = bboxes[..., 3] - bboxes[..., 1]
    
    bboxes[..., 0] = bboxes[..., 0] + w/2
    bboxes[..., 1] = bboxes[..., 1] + h/2
    bboxes[..., 2] = w
    bboxes[..., 3] = h
    
    return bboxes

def yolo2voc(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y2]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes

def coco2yolo(bboxes, image_height=720, image_width=1280):
    """
    coco => [xmin, ymin, w, h]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # normolizinig
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]/ image_height
    
    # converstion (xmin, ymin) => (xmid, ymid)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]/2
    
    return bboxes

def yolo2coco(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    coco => [xmin, ymin, w, h]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # denormalizing
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]* image_height
    
    # converstion (xmid, ymid) => (xmin, ymin) 
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    
    return bboxes

def voc2coco(bboxes, image_height=720, image_width=1280):
    bboxes  = voc2yolo(bboxes, image_height, image_width)
    bboxes  = yolo2coco(bboxes, image_height, image_width)
    return bboxes


def coco2voc(bboxes, image_height=720, image_width=1280):
    bboxes  = coco2yolo(bboxes, image_height, image_width)
    bboxes  = yolo2voc(bboxes, image_height, image_width)
    return bboxes


def load_image(image_path):
    return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)


def plot_one_box(x, img, color=None, label=None, line_thickness=None):
    # Plots one bounding box on image img
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

def draw_bboxes(img, bboxes, classes, class_ids, colors = None, show_classes = None, 
                bbox_format = 'yolo', class_name = False, line_thickness = 2):  
     
    image = img.copy()
    show_classes = classes if show_classes is None else show_classes
    colors = (0, 255 ,0) if colors is None else colors
    
    if bbox_format == 'yolo':
        
        for idx in range(len(bboxes)):  
            
            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            if cls in show_classes:
            
                x1 = round(float(bbox[0])*image.shape[1])
                y1 = round(float(bbox[1])*image.shape[0])
                w  = round(float(bbox[2])*image.shape[1]/2) #w/2 
                h  = round(float(bbox[3])*image.shape[0]/2)

                voc_bbox = (x1-w, y1-h, x1+w, y1+h)
                plot_one_box(voc_bbox, 
                             image,
                             color = color,
                             label = cls if class_name else str(get_label(cls)),
                             line_thickness = line_thickness)
            
    elif bbox_format == 'coco':
        
        for idx in range(len(bboxes)):  
            
            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            if cls in show_classes:            
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                w  = int(round(bbox[2]))
                h  = int(round(bbox[3]))

                voc_bbox = (x1, y1, x1+w, y1+h)
                plot_one_box(voc_bbox, 
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness)

    elif bbox_format == 'voc':
        
        for idx in range(len(bboxes)):  
            
            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            if cls in show_classes: 
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                x2 = int(round(bbox[2]))
                y2 = int(round(bbox[3]))
                voc_bbox = (x1, y1, x2, y2)
                plot_one_box(voc_bbox, 
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness)
    else:
        raise ValueError('wrong bbox format')

    return image

def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path'])
    return row

np.random.seed(32)
colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
          for idx in range(1)]


def decode_annotations(annotaitons_str):
    """decode annotations in string to list of dict"""
    return literal_eval(annotaitons_str)

def load_image_with_annotations(video_id, video_frame, image_dir, annotaitons_str):
    img = load_image(video_id, video_frame, image_dir)
    annotations = decode_annotations(annotaitons_str)
    if len(annotations) > 0:
        for ann in annotations:
            cv2.rectangle(img, (ann['x'], ann['y']),
                (ann['x'] + ann['width'], ann['y'] + ann['height']),
                (255, 0, 0), thickness=2,)
    return img

def draw_predictions(img, pred_bboxes):
    img = img.copy()
    if len(pred_bboxes) > 0:
        for bbox in pred_bboxes:
            conf = bbox[0]
            x, y, w, h = bbox[1:].round().astype(int)
            cv2.rectangle(img, (x, y),
                (x+w, y+h),
                (0, 255, 255), thickness=2,)
            cv2.putText(
                img,
                f"{conf:.2}",
                (x, max(0, y-5)),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 0, 255),
                thickness=1,
            )
    return img

# CLS model

In [15]:
class RegHead(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(RegHead, self).__init__()
        #self.norm = nn.LayerNorm(in_feat)
        self.fc1 = nn.Linear(in_feat, out_feat)
        #self.fc2 = nn.Linear(in_feat//2, out_feat)
        
        
    def forward(self, x):
        #x = self.norm(x)
        
        x = self.fc1(x)
        #x = F.relu_(x)
        #x = self.fc2(x)
        #x = x.sigmoid() * 100.
        return x

class PawModel(nn.Module):
    def __init__(self, arch='tf_efficientnet_b0_ns', pretrained=True, n_class=1):
        super(PawModel, self).__init__()
        
        self.model = timm.create_model(arch, pretrained=pretrained)
        
        #print(self.model); assert False
        
        try:
            if 'efficientnet' in  arch:
                self.fc = RegHead(self.model.classifier.in_features, 1)
                '''
                for k, v in self.model.named_parameters():
                    if 'blocks.4' in k or 'blocks.5' in k or 'blocks.6' in k:
                        print(f'{k} unfrozen')
                        v.requires_grad = True
                    else:
                        v.requires_grad = False
                '''        
                self.model.classifier = nn.Identity()
            elif 'swin' in arch or 'vit' in arch:
                self.fc = RegHead(self.model.head.in_features, n_class) #RegHead(self.model.head.in_features, 1)
                
                '''
                # 11-17
                
                unfreeze_kws = ['layers.2.blocks.16', 'layers.2.blocks.17', 'layers.2.downsample', 'layers.3']

                for k, v in self.model.named_parameters():
                    if any([kw in k for kw in unfreeze_kws]):
                        #print(f'{k} unfrozen')
                        v.requires_grad = True
                    else:
                        v.requires_grad = False
                '''  
                self.model.head = nn.Identity()
            else:
                self.fc = RegHead(self.model.fc.in_features, 1)
                self.model.fc = nn.Identity()
                
        except:
            print(self.model)
            #print(self.model.layers[3].blocks[1].mlp.fc2)
            assert False
            
    def forward(self, imgs):
        
        #with torch.no_grad():
        x = self.model(imgs) # (b*d, img_f)
            
        #x = F.normalize(x)
        
        x = self.fc(x)
        
        return x
    

# Metric Func

In [16]:
def calc_iou(bboxes1, bboxes2, bbox_mode='xywh'):
    # xmin ymin weight height
    assert len(bboxes1.shape) == 2 and bboxes1.shape[1] == 4
    assert len(bboxes2.shape) == 2 and bboxes2.shape[1] == 4
    
    bboxes1 = bboxes1.copy()
    bboxes2 = bboxes2.copy()
    
    if bbox_mode == 'xywh':
        bboxes1[:, 2:] += bboxes1[:, :2]
        bboxes2[:, 2:] += bboxes2[:, :2]

    x11, y11, x12, y12 = np.split(bboxes1, 4, axis=1)
    x21, y21, x22, y22 = np.split(bboxes2, 4, axis=1)
    xA = np.maximum(x11, np.transpose(x21))
    yA = np.maximum(y11, np.transpose(y21))
    xB = np.minimum(x12, np.transpose(x22))
    yB = np.minimum(y12, np.transpose(y22))
    interArea = np.maximum((xB - xA + 1), 0) * np.maximum((yB - yA + 1), 0)
    boxAArea = (x12 - x11 + 1) * (y12 - y11 + 1)
    boxBArea = (x22 - x21 + 1) * (y22 - y21 + 1)
    iou = interArea / (boxAArea + np.transpose(boxBArea) - interArea)
    return iou

def f_beta(tp, fp, fn, beta=2):
    return (1+beta**2)*tp / ((1+beta**2)*tp + beta**2*fn+fp+1e-6)

def calc_is_correct_at_iou_th(gt_bboxes, pred_bboxes, iou_th, verbose=False):
    gt_bboxes = gt_bboxes.copy()
    pred_bboxes = pred_bboxes.copy()
    
    tp = 0
    fp = 0
    for k, pred_bbox in enumerate(pred_bboxes): # fixed in ver.7
        ious = calc_iou(gt_bboxes, pred_bbox[None, 1:])
        max_iou = ious.max()
        if max_iou > iou_th:
            tp += 1
            gt_bboxes = np.delete(gt_bboxes, ious.argmax(), axis=0)
        else:
            fp += 1
        if len(gt_bboxes) == 0:
            fp += len(pred_bboxes) - (k + 1) # fix in ver.7
            break

    fn = len(gt_bboxes)
    return tp, fp, fn

def calc_is_correct(gt_bboxes, pred_bboxes):
    """
    gt_bboxes: (N, 4) np.array in xywh format
    pred_bboxes: (N, 5) np.array in conf+xywh format
    """
    if len(gt_bboxes) == 0 and len(pred_bboxes) == 0:
        tps, fps, fns = 0, 0, 0
        return tps, fps, fns
    
    elif len(gt_bboxes) == 0:
        tps, fps, fns = 0, len(pred_bboxes), 0
        return tps, fps, fns
    
    elif len(pred_bboxes) == 0:
        tps, fps, fns = 0, 0, len(gt_bboxes)
        return tps, fps, fns
    
    pred_bboxes = pred_bboxes[pred_bboxes[:,0].argsort()[::-1]] # sort by conf
    
    tps, fps, fns = 0, 0, 0
    for iou_th in np.arange(0.3, 0.85, 0.05):
        tp, fp, fn = calc_is_correct_at_iou_th(gt_bboxes, pred_bboxes, iou_th)
        tps += tp
        fps += fp
        fns += fn
    return tps, fps, fns

def calc_f2_score(gt_bboxes_list, pred_bboxes_list, verbose=False):
    """
    gt_bboxes_list: list of (N, 4) np.array in xywh format
    pred_bboxes_list: list of (N, 5) np.array in conf+xywh format
    """
    tps, fps, fns = 0, 0, 0
    for gt_bboxes, pred_bboxes in zip(gt_bboxes_list, pred_bboxes_list):
        tp, fp, fn = calc_is_correct(gt_bboxes, pred_bboxes)
        tps += tp
        fps += fp
        fns += fn
        if verbose:
            num_gt = len(gt_bboxes)
            num_pred = len(pred_bboxes)
            print(f'num_gt:{num_gt:<3} num_pred:{num_pred:<3} tp:{tp:<3} fp:{fp:<3} fn:{fn:<3}')
            
    print(f'tps:{tps}  fp:{fps}  fn:{fns}')
    return f_beta(tps, fps, fns, beta=2)

# Yolo & Tracker

In [91]:
def load_yolo_model(ckpt_path, conf=None, iou=None, max_det=None):
    model = torch.hub.load('./',
                           'custom',
                           path=ckpt_path,
                           source='local',
                           force_reload=True)  # local repo
    if conf is not None:
        model.conf = conf  # NMS confidence threshold
    if iou is not None:
        model.iou  = iou  # NMS IoU threshold
    model.classes = None   # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
    model.multi_label = False  # NMS multiple labels per box
    if model.max_det is not None:
        model.max_det = max_det  # maximum number of detections per image
    return model


def yolo_predict(model, img, conf, size=3600, augment=False):
    height, width = img.shape[:2]
    results = model(img, size=size, augment=augment)  # custom inference size
    preds   = results.pandas().xyxy[0]
    bboxes  = preds[['xmin','ymin','xmax','ymax']].values
    confs   = preds.confidence.values
    keep = confs > conf
    # voc format : 'xmin','ymin','xmax','ymax'
    return bboxes[keep], confs[keep]

    
def tracking_predict(tracker, frame_id, bboxes, scores):
    
    def to_norfair(detects, scores, frame_id):
        result = []
        for (x_min, y_min, x_max, y_max), score in zip(detects, scores):
            xc, yc = (x_min + x_max) / 2, (y_min + y_max) / 2
            w, h = x_max - x_min, y_max - y_min
            result.append(Detection(points=np.array([xc, yc]), scores=np.array([score]), data=np.array([w, h, frame_id])))

        return result
    
    track_bboxes = []
    track_scores = []
    
    tracked_objects = tracker.update(detections=to_norfair(bboxes, scores, frame_id))
    for tobj in tracked_objects:
        bbox_width, bbox_height, last_detected_frame_id = tobj.last_detection.data
        if last_detected_frame_id == frame_id:  # Skip objects that were detected on current frame
            continue
        xc, yc = tobj.estimate[0]
        score = tobj.last_detection.scores[0]
        
        track_bboxes.append([xc, yc, bbox_width, bbox_height])
        track_scores.append(score)
    track_bboxes = np.array(track_bboxes)
    track_scores = np.array(track_scores)
    
    if len(track_scores) == 0:
        return bboxes, scores
    
    track_bboxes[:, 0] = track_bboxes[:, 0] - track_bboxes[:, 2]/2
    track_bboxes[:, 1] = track_bboxes[:, 1] - track_bboxes[:, 3]/2
    track_bboxes = coco2voc(track_bboxes)

    if len(scores) == 0:
        return track_bboxes, track_scores

    bboxes = np.concatenate([bboxes, np.array(track_bboxes)], 0)
    scores = np.concatenate([scores, np.array(track_scores)], 0)

    return bboxes, scores


def cls_predict(cls_model, img, crops):
    
    conf = []
    
    with torch.no_grad():
        
        for crop in crops:
            
            x, y, x2, y2 = crop.astype(int)
            
            img_ = img[y:y2, x:x2, :]
            #print("crop", crop, img.shape, img_.shape)
            img_pt = tfm(image=img_)['image'].unsqueeze(0).to('cuda:0')
            p = cls_model(img_pt)[0].sigmoid().detach().cpu().numpy()[0]
            conf += [p]
            
    return np.array(conf)

    
def format_prediction(bboxes, confs):
    annot = ''
    if len(bboxes)>0:
        for idx in range(len(bboxes)):
            xmin, ymin, xmax, ymax = bboxes[idx]
            w = int(xmax - xmin)
            h = int(ymax - ymin)
            conf             = confs[idx]
            annot += f'{conf} {xmin} {ymin} {w} {h}'
            annot +=' '
        annot = annot.strip(' ')
    return annot


def show_img(img, bboxes, name, color, bbox_format='voc'):
    names  = [name]*len(bboxes)
    labels = [0]*len(bboxes)
    img    = draw_bboxes(img = img,
                           bboxes = bboxes, 
                           classes = names,
                           class_ids = labels,
                           class_name = True, 
                           colors = color, 
                           bbox_format = bbox_format,
                           line_thickness = 2)
    return img

In [92]:
def euclidean_distance(detection, tracked_object):
    return np.linalg.norm(detection.points - tracked_object.estimate)

In [93]:
def calc_f2_score_v2(gt_bboxes_list, pred_bboxes_list, verbose=False):
    """
    gt_bboxes_list: list of (N, 4) np.array in xywh format
    pred_bboxes_list: list of (N, 5) np.array in conf+xywh format
    """
    tps, fps, fns = 0, 0, 0
    for gt_bboxes, pred_bboxes in zip(gt_bboxes_list, pred_bboxes_list):
        tp, fp, fn = calc_is_correct(gt_bboxes, pred_bboxes)
        tps += tp
        fps += fp
        fns += fn
        if verbose:
            num_gt = len(gt_bboxes)
            num_pred = len(pred_bboxes)
            print(f'num_gt:{num_gt:<3} num_pred:{num_pred:<3} tp:{tp:<3} fp:{fp:<3} fn:{fn:<3}')
    
    precision = tps/(tps+fps+0.1)
    recall = tps/(tps+fns+0.1)

    f2 = f_beta(tps, fps, fns, beta=2)

    print(f'tps:{tps}  fps:{fps}  fns:{fns}, p: {precision:.3f}, r: {recall:.3f}, f2: {f2:.3f}')
    return f2

In [94]:
def get_bboxes_area(b):
    return (b[:, 2]-b[:, 0]).astype(int) * (b[:, 3]-b[:, 1]).astype(int)

In [104]:
def process_ilegal_bbox(pred_bbox, pred_conf):
    pred_bbox[:, 0] = np.clip(pred_bbox[:, 0], 0, 1200)
    pred_bbox[:, 2] = np.clip(pred_bbox[:, 2], 0, 1200)
    pred_bbox[:, 1] = np.clip(pred_bbox[:, 1], 0, 720)
    pred_bbox[:, 3] = np.clip(pred_bbox[:, 3], 0, 720)
    pred_bbox = np.clip(pred_bbox, 0, 1200)
    area = get_bboxes_area(pred_bbox)
    keep = np.logical_and(area > MIN_AREA, area < MAX_AREA)
    pred_bbox = pred_bbox[keep]
    pred_conf = pred_conf[keep]
    
    return pred_bbox, pred_conf

In [105]:
c1 = (np.random.randint(255), np.random.randint(255), np.random.randint(255))
c2 = (np.random.randint(255), np.random.randint(255), np.random.randint(255))


def evaluate(model, cls_model, tracker, df, name, **kw):
    # output video config
    frame_id = 0
    fps = 24 # don't know exact value
    width = 1280
    height = 720
    video_path = ROOT+name
    print("Video save in :", video_path)
    output_video = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*"MP4V"), fps, (1280, height))
    
    gt_bbox_list = []
    pr_bbox_list = []
    pr_conf_list = []
    for i, row in tqdm(df.iterrows()):
        image = load_image(row.image_path)
        pred_bbox, pred_conf = yolo_predict(model, image, INFER_CONF, IMG_SIZE, AUGMENT)
        if tracker is not None:
            pred_bbox, pred_conf = tracking_predict(tracker, frame_id, pred_bbox, pred_conf)
        
        pred_bbox, pred_conf = process_ilegal_bbox(pred_bbox, pred_conf)
        #print(len(pred_bbox), pred_bbox)
        if len(pred_conf) > 0:
            if cls_model is not None:
                # print(image.shape, pred_bbox)
                cls_confs = cls_predict(cls_model, image, pred_bbox)
                filt = cls_confs > CLS_CONF
                pred_bbox = pred_bbox[filt]
                pred_conf = pred_conf[filt]
            pred_bbox = voc2coco(pred_bbox)
            pred = np.concatenate((pred_conf[..., None], pred_bbox), axis=1)
        else:
            pred = np.array([])
            
        gt = np.array(row.bboxes)
        gt_bbox_list.append(gt)
        
        pr_bbox_list.append(pred)
        pr_conf_list.append(pred_conf)
#         print("GT", gt)
#         print("PR", pred)
#         print("-"*50)
        
        img = load_image(row.image_path)[:,:,::-1]
        plt.figure(figsize=(16, 12))
        img = show_img(img, pred_bbox, 'PRED', c1, 'coco')
        img = show_img(img, gt, 'GT', c2, 'coco')
        output_video.write(img)
        plt.close()
        frame_id += 1
        #if i > 100:
        #    break

    # perform iou and conf search for best f2
    
    
    score = calc_f2_score_v2(gt_bbox_list, pr_bbox_list, verbose=False)
    print(f"F2 score {score: .5f}")
    print(f"Video save in {video_path}")
    return score

In [107]:
tfm = A.Compose([A.Resize(224, 224, interpolation=cv2.INTER_LANCZOS4),      
                A.Normalize(mean=[0], std=[1], max_pixel_value=255.0, p=1.0),
                ToTensorV2(p=1.0),
            ], p=1.0)
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
device = 'cuda:0'
cls_model = PawModel('swin_large_patch4_window7_224', pretrained=False, n_class=1).to(device)
ckpt = './checkpoints/swin224-cls-fold_1_ep_2'
cls_model.load_state_dict(torch.load(ckpt, map_location=device))
cls_model.eval()


ROOT  = './cv/'
Path(ROOT).mkdir(exist_ok=True, parents=True)
CKPT_PATH = './starfish/yolov5s6_e5_bs2_img5400_video1_HRv1.1_01-21/weights/yolov5s6_e5_bs2_lr01_img5400_public_video1.pt'
IMG_SIZE  = 5400
MNS_CONF      = 0.01  # 0.25
MNS_IOU       = None  # 0.45
INFER_CONF = 0.01
CLS_CONF   = 0.9
MAX_DET   = 1000  #1000
AUGMENT   = False
MIN_AREA  = 240
MAX_AREA  = 20000
VIDEO = 1

DATA  = f'../input/yolov5_video{VIDEO}/train.csv'
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

df = pd.read_csv(DATA)
df['annotations'] = df['annotations'].apply(lambda x: ast.literal_eval(x))
df['num_bbox'] = df['annotations'].apply(lambda x: len(x))
val = df[df.fold == 'val'].reset_index(drop=True)
val['bboxes'] = val['bboxes'].apply(eval)
val = val.sort_values(['video_id', 'video_frame'])

model = load_yolo_model(CKPT_PATH, MNS_CONF, MNS_IOU, MAX_DET)
tracker = Tracker(
    distance_function=euclidean_distance, 
    distance_threshold=30,
    hit_inertia_min=3,
    hit_inertia_max=6,
    initialization_delay=1,
)
video_name = CKPT_PATH.split("/")[-1][:-3]+ "_conf0.05_"+".mp4"
evaluate(model, cls_model, tracker, val, video_name)

YOLOv5 🚀 4b1ec65 torch 1.9.1+cu111 CUDA:0 (NVIDIA GeForce RTX 3090, 24268MiB)

Fusing layers... 
Model Summary: 280 layers, 12308200 parameters, 0 gradients, 16.2 GFLOPs
Adding AutoShape... 
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Video save in : ./cv/yolov5s6_e5_bs2_lr01_img5400_public_video1_conf0.05_.mp4


0it [00:00, ?it/s]

tps:43865  fps:54252  fns:25329, p: 0.447, r: 0.634, f2: 0.585
F2 score  0.58503
Video save in ./cv/yolov5s6_e5_bs2_lr01_img5400_public_video1_conf0.05_.mp4


0.585033596251237